In [31]:
from Bio import SeqIO, Entrez
import os


In [32]:
def affichage(sequence,seg_list):
    print(f"id :{sequence.id}")
    print(f"longeur : {len(sequence.seq)}")
    seg_list.append(sequence.seq)
    print(f"séquence : {sequence.seq}")

In [33]:
def explore_alignments(i, j, current_align1, current_align2,alignments,score_matrix,seq1,seq2):
    # Condition d'arrêt
    if i < 2 and j < 2 :
        alignments.append((current_align1, current_align2))
        return

    diag = score_matrix[i - 1][j - 1] if i > 0 and j > 0 else float('-inf')
    up = score_matrix[i - 1][j] if i > 0 else float('-inf')
    left = score_matrix[i][j - 1] if j > 0 else float('-inf')

    max_value = max(diag, up, left)
    if max_value == diag:
        if i>1 and j>1 :
        
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)
        elif j<1 : 
            explore_alignments(i - 1, j - 1, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
        elif i<1:
             explore_alignments(i - 1, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    if max_value == up:
        explore_alignments(i - 1, j, seq1[i -2] + current_align1, '-' + current_align2,alignments,score_matrix,seq1,seq2)
    if max_value == left:
        explore_alignments(i, j - 1, '-' + current_align1, seq2[j -2] + current_align2,alignments,score_matrix,seq1,seq2)

    return alignments    


In [34]:
def global_alignment(seq1, seq2, match=2, mismatch=-1, gap=-1):

    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]
    print(score_matrix)
    # traceback_matrix = [[None] * m for _ in range(n)]

    # Rempli gap
    for i in range(1, n):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap
        # traceback_matrix[i][0] = 'up'
    for j in range(1, m):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap
        # traceback_matrix[0][j] = 'left'


    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            #best score
            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            max_score = max(diag, up, left)
            score_matrix[i][j] = max_score

            # if max_score == diag:
            #     traceback_matrix[i][j] = 'diag'
            # elif max_score == up:
            #     traceback_matrix[i][j] = 'up'
            # else:
            #     traceback_matrix[i][j] = 'left'

    alignments = []  
    i, j = n - 1, m - 1
    initial_align1 = seq1[i-1]  
    initial_align2 = seq2[j-1]  

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
                    
    print("Matrice de score :")
    for row in score_matrix:
        print(row)    
    
    return alignments, score_matrix[n - 1][m - 1]




In [35]:
def local_alignment(seq1, seq2, match, mismatch, gap):
    n = len(seq1) + 1
    m = len(seq2) + 1

    score_matrix = [[0] * m for _ in range(n)]

    # max_pos = (0, 0)
    
    for i in range(1, n):
        for j in range(1, m):
            if seq1[i - 1] == seq2[j - 1]:
                score = match
            else:
                score = mismatch

            diag = score_matrix[i - 1][j - 1] + score
            up = score_matrix[i - 1][j] + gap
            left = score_matrix[i][j - 1] + gap
            
            score_matrix[i][j] = max(0,diag, up, left)
            
            # # Track maximum score position
            # if score_matrix[i][j] > max_score:
            #     max_score = score_matrix[i][j]

    End = score_matrix[n - 1][m - 1]
    diag = score_matrix[n - 2][m - 2]
    up = score_matrix[n - 2][m-1]
    left = score_matrix[n-1][m - 2]
    score_final = max(diag,up,left,End) 

    if  score_final == End:
        i, j = n - 1, m - 1
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-1]
    elif   score_final == diag:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-2]        
    elif   score_final == up:
        i, j = n - 2, m - 1
        initial_align1 = seq1[i-2]  
        initial_align2 = seq2[j-1]
    elif score_final == left: 
        i, j = n - 1, m - 2 
        initial_align1 = seq1[i-1]  
        initial_align2 = seq2[j-2]   


    alignments = []  
      

    alignments = explore_alignments(i, j, initial_align1, initial_align2,alignments,score_matrix,seq1,seq2)
       
    print("Matrice de score :")
    for row in score_matrix:
        print(row)
    
    return alignments, score_final


In [36]:
def calculate_score(align1, align2, match, mismatch, gap):
    score = 0
    for a, b in zip(align1, align2):
        if a == '-' or b == '-':
            score += gap
        elif a == b:
            score += match
        else:
            score += mismatch
    return score

In [37]:
# Exemple d'utilisation
seq1 = "CTTG"
seq2 = "ACTGG"
alignments, score = global_alignment(seq1, seq2,2,-1,-1)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-1)}' ) for align1, align2 in alignments]
print(scored_alignments)


[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
Matrice de score :
[0, -1, -2, -3, -4, -5]
[-1, -1, 1, 0, -1, -2]
[-2, -2, 0, 3, 2, 1]
[-3, -3, -1, 2, 2, 1]
[-4, -4, -2, 1, 4, 4]
Score de la matrice : 4
Alignement de la séquence 1 : [('-CTT-G', 'AC-TGG'), ('-CTT-G', 'ACT-GG')]
[('-CTT-G', 'AC-TGG', 'score = 3'), ('-CTT-G', 'ACT-GG', 'score = 3')]


In [38]:
alignments, score = local_alignment(seq1, seq2,2,-1,-2)
print("Score de la matrice :", score)
print("Alignement de la séquence 1 :", alignments)
scored_alignments = [(align1, align2, f'score = {calculate_score(align1, align2,2,-1,-2)}' ) for align1, align2 in alignments]
print(scored_alignments)


Matrice de score :
[0, 0, 0, 0, 0, 0]
[0, 0, 2, 0, 0, 0]
[0, 0, 0, 4, 2, 0]
[0, 0, 0, 2, 3, 1]
[0, 0, 0, 0, 4, 5]
Score de la matrice : 5
Alignement de la séquence 1 : [('-GCTT-G', 'A-CT-GG'), ('-CTT-G', 'ACT-GG')]
[('-GCTT-G', 'A-CT-GG', 'score = -2'), ('-CTT-G', 'ACT-GG', 'score = 0')]


In [39]:
import numpy as np
import math
from Bio.pairwise2 import align

# Étape 1 : Calcul des Scores d'Alignement Pair-à-Pair
def global_alignment(seq1, seq2):
    alignment = align.globalxx(seq1, seq2)
    score = alignment[0].score
    return alignment, score

def calculate_pairwise_scores(sequences):
    num_seqs = len(sequences)
    scores_matrix = np.zeros((num_seqs, num_seqs))

    for i in range(num_seqs):
        for j in range(i + 1, num_seqs):
            _, score = global_alignment(sequences[i], sequences[j])
            scores_matrix[i][j] = scores_matrix[j][i] = score

    return scores_matrix

# Étape 2 : Conversion des Scores en Distances
def convert_to_distance_matrix(scores_matrix):
    num_seqs = scores_matrix.shape[0]
    distance_matrix = np.zeros_like(scores_matrix)

    for i in range(num_seqs):
        for j in range(i + 1, num_seqs):
            max_score = max(scores_matrix[i][i], scores_matrix[j][j])
            if max_score == 0:
                p = 0
            else:
                p = 1 - (scores_matrix[i][j] / max_score)
            if 1 - 2 * p - (p ** 2) <= 0:
                distance = float('inf')
            else:
                distance = -0.5 * math.log(1 - 2 * p - (p ** 2))
            distance_matrix[i][j] = distance_matrix[j][i] = distance

    return distance_matrix

# Étape 3 : Construction de l'Arbre Guide
def construct_guide_tree(distance_matrix):
    num_seqs = len(distance_matrix)
    clusters = {i: [i] for i in range(num_seqs)}
    guide_tree = []

    while len(clusters) > 1:
        # Trouver les deux clusters les plus proches
        min_dist = float('inf')
        closest_pair = None

        for i in clusters:
            for j in clusters:
                if i < j:
                    dist = np.mean([distance_matrix[x][y] for x in clusters[i] for y in clusters[j]])
                    if dist < min_dist:
                        min_dist = dist
                        closest_pair = (i, j)

        # Fusionner les deux clusters les plus proches
        i, j = closest_pair
        new_cluster = clusters[i] + clusters[j]
        guide_tree.append((clusters[i], clusters[j]))

        # Supprimer les anciens clusters
        del clusters[i]
        del clusters[j]

        # Ajouter le nouveau cluster
        new_cluster_id = max(clusters.keys(), default=-1) + 1
        clusters[new_cluster_id] = new_cluster

        # Redimensionner la matrice de distance
        new_size = len(distance_matrix) + 1
        new_distance_matrix = np.zeros((new_size, new_size))
        new_distance_matrix[:len(distance_matrix), :len(distance_matrix)] = distance_matrix

        # Mettre à jour la matrice de distance pour le nouveau cluster
        for k in clusters:
            if k != new_cluster_id:
                dist_to_new = (np.mean([new_distance_matrix[x][y] for x in new_cluster for y in clusters[k]]))
                new_distance_matrix[k][new_cluster_id] = dist_to_new
                new_distance_matrix[new_cluster_id][k] = dist_to_new

        # Mettre à jour la matrice de distance
        distance_matrix = new_distance_matrix

    return guide_tree


# Étape 4 : Alignement Multiple Progressif
def progressive_alignment(sequences, guide_tree):
    # Aligner les séquences ou les clusters en suivant l'arbre guide
    aligned_sequences = [sequences[i] for i in range(len(sequences))]

    for cluster1, cluster2 in guide_tree:
        # Aligner les séquences des deux clusters
        seq1 = ''.join([aligned_sequences[i] for i in cluster1])
        seq2 = ''.join([aligned_sequences[i] for i in cluster2])
        alignment, _ = global_alignment(seq1, seq2)

        # Mettre à jour les alignements pour chaque cluster
        for i, seq in zip(cluster1, alignment[0].seqA):
            aligned_sequences[i] = seq
        for i, seq in zip(cluster2, alignment[0].seqB):
            aligned_sequences[i] = seq

    return aligned_sequences

# Exécution
sequences = ["AGTAC", "GATC", "ATGC"]
scores_matrix = calculate_pairwise_scores(sequences)
distance_matrix = convert_to_distance_matrix(scores_matrix)
guide_tree = construct_guide_tree(distance_matrix)

print("Matrice de Scores d'Alignement :")
print(scores_matrix)
print("Matrice de Distances :")
print(distance_matrix)
print("Arbre Guide :")
print(guide_tree)

# Aligner les séquences avec l'arbre guide
aligned_sequences = progressive_alignment(sequences, guide_tree)
print("Alignement Multiple Final :")
print(aligned_sequences)


Matrice de Scores d'Alignement :
[[0. 3. 3.]
 [3. 0. 3.]
 [3. 3. 0.]]
Matrice de Distances :
[[ 0. -0. -0.]
 [-0.  0. -0.]
 [-0. -0.  0.]]
Arbre Guide :
[([0], [1]), ([2], [0, 1])]
Alignement Multiple Final :
['A', '-', 'A']
